In [86]:
#importing libaries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [ ]:
# Function to extract Product Title
def get_title(soup):
    try:
        # Find the HTML element containing the product title
        title = soup.find("span", attrs={"id": 'productTitle'})
        
        # Extract the text value from the element
        title_value = title.text

        # Remove any extra whitespace from the title
        title_string = title_value.strip()

    except AttributeError:
        # If the product title is not found, return an empty string
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):
    try:
        # Find the standard price of the product
        price = soup.find("span", attrs={'id': 'priceblock_ourprice'}).string.strip()

    except AttributeError:
        # If standard price is not found, check for a deal price
        try:
            price = soup.find("span", attrs={'id': 'priceblock_dealprice'}).string.strip()

        except:
            # If no price is found, return an empty string
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):
    try:
        # Find the rating with specific class
        rating = soup.find("i", attrs={'class': 'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        # If the first rating format is not found, check another format
        try:
            rating = soup.find("span", attrs={'class': 'a-icon-alt'}).string.strip()
        except:
            # If no rating is found, return an empty string
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        # Find the element containing the review count
        review_count = soup.find("span", attrs={'id': 'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        # If review count is not found, return an empty string
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        # Find the element indicating product availability
        available = soup.find("div", attrs={'id': 'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        # If availability information is not found, default to "Not Available"
        available = "Not Available"	

    return available


In [ ]:

if __name__ == '__main__':

    # Add your User-Agent string to avoid getting blocked by Amazon
    HEADERS = ({
        'User-Agent': '',  # Replace with your User-Agent string
        'Accept-Language': 'en-US, en;q=0.5'
    })

    # URL of the Amazon search results page
    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"

    # Make an HTTP GET request to fetch the webpage content
    webpage = requests.get(URL, headers=HEADERS)

    # Create a BeautifulSoup object to parse the HTML content
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Find all links to product pages
    links = soup.find_all("a", attrs={'class': 'a-link-normal s-no-outline'})

    # Store the extracted links in a list
    links_list = []
    for link in links:
        links_list.append(link.get('href'))

    # Dictionary to store product details
    d = {"title": [], "price": [], "rating": [], "reviews": [], "availability": []}
    
    # Loop through each product link to extract details
    for link in links_list:
        # Make an HTTP GET request for the product page
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        # Create a BeautifulSoup object for the product page
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Extract and store product details using the defined functions
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    # Convert the dictionary to a DataFrame
    amazon_df = pd.DataFrame.from_dict(d)

    # Replace empty titles with NaN for better data cleaning
    amazon_df['title'] = amazon_df['title'].replace('', np.nan)

    # Drop rows where the title is missing
    amazon_df = amazon_df.dropna(subset=['title'])

    # Save the DataFrame to a CSV file
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


In [104]:
amazon_df

,title,price,rating,reviews,availability
0,PlayStation 4 Slim 1TB Console,,4.7 out of 5 stars,"15,738 ratings",In Stock
1,PlayStation®4 Console – Call of Duty® Modern W...,,4.5 out of 5 stars,118 ratings,In Stock
2,$250 PlayStation Store Gift Card [Digital Code],,4.6 out of 5 stars,"257,540 ratings",Available now
3,Sony Playstation Classic Console with 20 Plays...,,4.1 out of 5 stars,"2,194 ratings",Only 8 left in stock - order soon.
4,"PS4 Controller Charger Dock Station, OIVO 1.8H...",,4.7 out of 5 stars,"39,498 ratings",In Stock
5,EA SPORTS FC 25 - PlayStation 4,,4.7 out of 5 stars,13 ratings,In Stock
6,"Wuthur PS4 Controller Wireless 2 Pack, Compati...",,4.3 out of 5 stars,190 ratings,In Stock
7,Seagate (STGD2000100) Game Drive for PS4 Syste...,,4.7 out of 5 stars,"29,915 ratings",
8,"FASIGO PS4 Controller 2 Pack, Wireless PS4 Con...",,4.2 out of 5 stars,481 ratings,In Stock
9,Madden NFL 25 - PlayStation 4,,4.2 out of 5 stars,60 ratings,In Stock


In [92]:
amazon_df.to_excel("amazon_data.xlsx", index=False)

In [101]:
# prompt: download the amazon data.xlsx from jupyter notebook and google colab
from IPython.display import FileLink

FileLink('amazon_data.xlsx')


C:\Users\nikhi\Downloads\amazon_data.xlsx